In [2]:
import os
os.environ["HF_DATASETS_OFFLINE"] = "1"
from datasets import load_dataset


ds = load_dataset("Team-ACE/ToolACE")
ds

DatasetDict({
    train: Dataset({
        features: ['system', 'conversations'],
        num_rows: 11300
    })
})

In [115]:
import json
import re
import json


pattern = r'([a-zA-Z0-9_\'\(\)\-\.\/ ]+)\(([^)]*?)\)'

filter_fname_chars = set(['#', '&', '+', ',', ':', '?', '@',  '[', ']', '{', '|', '}', 'ã', 'é', 'ñ', 'ö', 'ı'])


def convert(example):
    ret = {}
    tools = re.search(r'JSON format that you can invoke:\n(.*?). \nShould you decide to', example["system"], re.DOTALL).group(1)
    tools = [{"type": "function", "function": f} for f in json.loads(tools)]
    for t in tools:
        if set(t["function"]["name"]) & filter_fname_chars:
            raise
    if not tools:
        raise
    ret["tools"] = json.dumps(tools, ensure_ascii=False)

    messages = []
    for c in example["conversations"]:
        if c["from"] == "user":
            messages.append({"role": "user", "content": c["value"]})
        elif c["from"] == "assistant":
            if c["value"].startswith("[") and c["value"].endswith("]"):
                tool_calls = re.findall(pattern, c["value"])
                tool_calls = [{
                    "type": "function", 
                    "function": {
                        "name": t[0].strip(), 
                        "arguments": json.dumps(eval(f"dict({t[1]})"), ensure_ascii=False)}
                } for t in tool_calls]
                messages.append({"role": "assistant", "tool_calls": tool_calls})
            else:
                messages.append({"role": "assistant", "content": c["value"]})
        elif c["from"] == "tool":
            for v in json.loads(c["value"]):
                messages.append({"role": "tool", "content": json.dumps(v, ensure_ascii=False)})
    ret["conversation"] = messages

    return ret

datas = []
for d in ds["train"]:
    try:
        datas.append(convert(d))
    except:
        pass

len(datas)

10086

In [123]:
replace_fname_chars = set([' ', "'", '(', ')', '-', '.', '/'])

def fix_schema(schema):
    if schema["type"] == "dict":
        schema["type"] = "object"
    if schema["type"] == "int":
        schema["type"] = "integer"
    if schema["type"] == "float":
        schema["type"] = "number"
    if schema["type"] == "array":
        schema["items"] = fix_schema(schema["items"])
    if "properties" in schema:
        schema["properties"] = {k: fix_schema(v) for k, v in schema["properties"].items()}
    return schema
        
def convert_func_name(data):
    tools = json.loads(data["tools"])
    for t in tools:
        t["function"]["parameters"] = fix_schema(t["function"]["parameters"])
        if chars := set(t["function"]["name"]) & replace_fname_chars:
            for ch in chars:
                t["function"]["name"] = t["function"]["name"].replace(ch, "_")
    
    for m in data["conversation"]:
        if tool_calls := m.get("tool_calls"):
            for t in tool_calls:
                if chars := set(t["function"]["name"]) & replace_fname_chars:
                    for ch in chars:
                        t["function"]["name"] = t["function"]["name"].replace(ch, "_")
        if m["role"] == "tool":
            content = json.loads(m["content"])
            if chars := set(content["name"]) & replace_fname_chars:
                for ch in chars:
                    content["name"] = content["name"].replace(ch, "_")
                m["content"] = json.dumps(content, ensure_ascii=False)
            
    data["tools"] = json.dumps(tools, ensure_ascii=False)
    return data


for d in datas:
    convert_func_name(d)

datas[0]

{'tools': '[{"type": "function", "function": {"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can\'t restore access to an address if you lose it.", "parameters": {"type": "object", "properties": {"password": {"description": "The password for the new Ethereum address", "type": "string"}}, "required": ["password"]}, "required": null}}, {"type": "function", "function": {"name": "Market_Trends_API", "description": "Get the latest market trends and relevant news for a specified country and language.", "parameters": {"type": "object", "properties": {"trend_type": {"description": "Trend type.", "type": "string", "enum": ["MARKET_INDEXES", "MOST_ACTIVE", "GAINERS", "LOSERS", "CRYPTO", "CURRENCIES", "CLIMATE_LEADERS"]}, "country": {"description": "The country for which to get trends, specified as a 2-letter country code - see ISO 3166.", "type": "string", "default": "us"}, "language": {"description": 

In [127]:
import checker_oai
import importlib
importlib.reload(checker_oai)
ck = checker_oai.Checker()

from collections import defaultdict

filter_datas = []
for d in datas:
    if r := ck.check(d) is None:
        filter_datas.append(d)

len(filter_datas)


ERROR:root:'float' is not valid under any of the given schemas

Failed validating 'anyOf' in metaschema['allOf'][1]['properties']['properties']['additionalProperties']['$dynamicRef']['allOf'][1]['properties']['properties']['additionalProperties']['$dynamicRef']['allOf'][1]['properties']['additionalProperties']['$dynamicRef']['allOf'][3]['properties']['type']:
    {'anyOf': [{'$ref': '#/$defs/simpleTypes'},
               {'type': 'array',
                'items': {'$ref': '#/$defs/simpleTypes'},
                'minItems': 1,
                'uniqueItems': True}]}

On schema['properties']['initial_population']['properties']['allele_frequencies']['additionalProperties']['type']:
    'float'
Traceback (most recent call last):
  File "/Users/admin/fish/octupus-tool-call/data/checker_oai.py", line 75, in check
    self.check_schema(data)
  File "/Users/admin/fish/octupus-tool-call/data/checker_oai.py", line 59, in check_schema
    raise e
  File "/Users/admin/fish/octupus-tool-call/data/ch

{'initial_population': {'population_id': 'NA_Wolves', 'allele_frequencies': {'alleleA': 0.3, 'alleleB': 0.7}}, 'simulation_parameters': {'number_of_generations': 10, 'generation_time': 5, 'population_size': 150}} {'type': 'object', 'properties': {'initial_population': {'description': 'Initial genetic composition of the population.', 'type': 'object', 'properties': {'population_id': {'description': 'Unique identifier for the population.', 'type': 'string'}, 'allele_frequencies': {'description': 'Initial frequencies of different alleles in the population.', 'type': 'object', 'additionalProperties': {'type': 'float'}}}, 'required': ['population_id', 'allele_frequencies']}, 'simulation_parameters': {'description': 'Parameters for the genetic drift simulation.', 'type': 'object', 'properties': {'number_of_generations': {'description': 'Number of generations to simulate.', 'type': 'number'}, 'generation_time': {'description': 'Time in years for one generation.', 'type': 'number'}, 'populatio

ERROR:root:'float' is not valid under any of the given schemas

Failed validating 'anyOf' in metaschema['allOf'][1]['properties']['properties']['additionalProperties']['$dynamicRef']['allOf'][1]['properties']['properties']['additionalProperties']['$dynamicRef']['allOf'][1]['properties']['additionalProperties']['$dynamicRef']['allOf'][3]['properties']['type']:
    {'anyOf': [{'$ref': '#/$defs/simpleTypes'},
               {'type': 'array',
                'items': {'$ref': '#/$defs/simpleTypes'},
                'minItems': 1,
                'uniqueItems': True}]}

On schema['properties']['initial_population']['properties']['allele_frequencies']['additionalProperties']['type']:
    'float'
Traceback (most recent call last):
  File "/Users/admin/fish/octupus-tool-call/data/checker_oai.py", line 75, in check
    self.check_schema(data)
  File "/Users/admin/fish/octupus-tool-call/data/checker_oai.py", line 59, in check_schema
    raise e
  File "/Users/admin/fish/octupus-tool-call/data/ch

{'initial_population': {'population_id': 'population123', 'allele_frequencies': {'A': 0.5, 'B': 0.5}}, 'simulation_parameters': {'number_of_generations': 100.0, 'generation_time': 1.0, 'population_size': 1000.0}} {'type': 'object', 'properties': {'initial_population': {'description': 'Initial genetic composition of the population.', 'type': 'object', 'properties': {'population_id': {'description': 'Unique identifier for the population.', 'type': 'string'}, 'allele_frequencies': {'description': 'Initial frequencies of different alleles in the population.', 'type': 'object', 'additionalProperties': {'type': 'float'}}}, 'required': ['population_id', 'allele_frequencies']}, 'simulation_parameters': {'description': 'Parameters for the genetic drift simulation.', 'type': 'object', 'properties': {'number_of_generations': {'description': 'Number of generations to simulate.', 'type': 'number'}, 'generation_time': {'description': 'Time in years for one generation.', 'type': 'number'}, 'populatio

10080

In [128]:
from collections import defaultdict
function_stat = defaultdict(lambda: 0)
for sample in filter_datas:
    if not sample["tools"]:
        function_stat[None] += 1
    else:
        functions = json.loads(sample["tools"]) 
        for f in functions:
            function_stat[f["function"]["name"]] += 1

sorted_stat = sorted(function_stat.items(), key=lambda x: x[1], reverse=True)
print(sorted_stat)
len(sorted_stat)

[('Search', 74), ('Get_Supported_Languages', 47), ('Get_Products', 36), ('loginUser', 34), ('Get_Products_in_Category', 33), ('getUserByName', 31), ('Get_User_Information', 30), ('Get_Categories', 27), ('Get_User_Followers', 26), ('GetFileVersions', 24), ('getInventory', 24), ('getOrderById', 23), ('GetFilesList', 22), ('Get_Countries', 21), ('Get_Product', 21), ('Autocomplete', 20), ('Measure_Distance', 19), ('Generate_QR_Code', 19), ('search', 19), ('US_Gas_Prices_API', 19), ('Facilities_Lookup', 18), ('Get_Product_Reviews', 18), ('Get_Product_Details', 18), ('Get_Sports_List', 18), ('MatchSchedules', 18), ('Get_User_Prostring', 18), ('Get_Order', 18), ('Historical_Stock_Prices', 16), ('Get_User_Posts', 16), ('auto_complete', 16), ('Get_Property_Details', 16), ('US_Gas_Prices', 15), ('Get_Languages', 15), ('Get_Cities', 15), ('StorageExists', 15), ('Search_Users', 14), ('Send_SMS', 14), ('IP_Lookup', 14), ('Get_UFC_Fight_Details', 13), ('Currency_Converter', 13), ('Get_Users', 13), (

15599

In [129]:
from datasets import Dataset
Dataset.from_list(filter_datas).push_to_hub("hqfx/tool_ace_cleaned")

Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.41s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/hqfx/tool_ace_cleaned/commit/90e4bc4752dbcf84bd751d80cd57298e42f36bbe', commit_message='Upload dataset', commit_description='', oid='90e4bc4752dbcf84bd751d80cd57298e42f36bbe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hqfx/tool_ace_cleaned', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hqfx/tool_ace_cleaned'), pr_revision=None, pr_num=None)